In [ ]:
# Install required packages
!pip install flask
!pip install pyngrok
!pip install flask-cors
!pip install tensorflow==2.15.0  # Ensure TensorFlow is installed if not already

# Import necessary libraries
from google.colab import drive
import pickle
import numpy as np
import pandas as pd
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from pyngrok import ngrok
import time

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory in your Google Drive where your files are saved
drive_project_path = '/content/drive/MyDrive/pattern_recognition/HM2/output/'

# Load the model
model_load_path = os.path.join(drive_project_path, 'sentiment_analysis_model.h5')
model = load_model(model_load_path)
print("Model loaded successfully.")

# Configure Ngrok
ngrok_auth_token = "2o6wYIHtqQ7pCho4BtgWVRn93kP_33xXCQg3S5t3WvBZocWya"
ngrok.set_auth_token(ngrok_auth_token)

# Open a tunnel to the Flask app on port 5000
public_url = ngrok.connect(5173)
print(f"Public URL: {public_url}")

# Load the tokenizer
tokenizer_load_path = os.path.join(drive_project_path, 'tokenizer.pickle')
with open(tokenizer_load_path, 'rb') as handle:
    tokenizer = pickle.load(handle)
print("Tokenizer loaded successfully.")

# Load max_seq_length
max_seq_length_load_path = os.path.join(drive_project_path, 'max_seq_length.pickle')
with open(max_seq_length_load_path, 'rb') as handle:
    max_seq_length = pickle.load(handle)
print("max_seq_length loaded successfully.")

# Define the text cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub('https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Create a Flask app instance
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/', methods=['GET'])
def home():
    return "Welcome to the Sentiment Analysis API."

@app.route('/sentiment', methods=['GET'])
def sentiment_info():
    return "This endpoint accepts POST requests with JSON payload {'text': 'your text'}."

# Define the sentiment prediction route
@app.route('/sentiment', methods=['POST'])
def predict_sentiment():
    try:
        data = request.get_json()
        text = data['text']

        # Validate input
        if not text:
            return jsonify({'error': 'No text provided.'}), 400

        # Clean the review
        cleaned_review = clean_text(text)

        # Convert to sequence
        review_sequence = tokenizer.texts_to_sequences([cleaned_review])

        # Pad the sequence
        review_padded = pad_sequences(review_sequence, maxlen=max_seq_length, padding='post', truncating='post')

        # Predict sentiment
        prediction = model.predict(review_padded)[0][0]

        # Determine sentiment and probability
        sentiment = "Positive" if prediction > 0.5 else "Negative"
        probability = prediction if prediction > 0.5 else 1 - prediction

        return jsonify({'sentiment': sentiment, 'probability': float(probability)})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the Flask app
# Bind to '0.0.0.0' to make it accessible externally
app.run(host="0.0.0.0", port=5173)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model loaded successfully.
Public URL: NgrokTunnel: "https://8def-34-80-240-89.ngrok-free.app" -> "http://localhost:5173"
Tokenizer loaded successfully.
max_seq_length loaded successfully.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5173
 * Running on http://172.28.0.12:5173
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 00:32:44] "GET /sentiment HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 00:32:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 00:33:08] "GET /sentiment HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 00:33:51] "OPTIONS /sentiment HTTP/1.1" 200 -


1/1 [==============================] - 1s 653ms/step


INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 00:33:52] "POST /sentiment HTTP/1.1" 200 -
